In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import string
import nltk
from nltk.corpus import stopwords

In [3]:
df=pd.read_csv('data.csv')
df

,Sentence,Sentiment
0,The GeoSolutions technology will leverage Bene...,positive
1,"$ESI on lows, down $1.50 to $2.50 BK a real po...",negative
2,"For the last quarter of 2010 , Componenta 's n...",positive
3,According to the Finnish-Russian Chamber of Co...,neutral
4,The Swedish buyout firm has sold its remaining...,neutral
...,...,...
5837,RISING costs have forced packaging producer Hu...,negative
5838,Nordic Walking was first used as a summer trai...,neutral
5839,"According shipping company Viking Line , the E...",neutral
5840,"In the building and home improvement trade , s...",neutral


In [4]:
df.isnull().sum()

Sentence     0
Sentiment    0
dtype: int64

In [43]:

def cleaning(text):
    data="".join([d for d in text if d not in string.punctuation])
    return [i for i in data.split() if i not in stopwords.words('english')]

In [52]:
from sklearn.feature_extraction.text import CountVectorizer

In [61]:
bow_fit=CountVectorizer(analyzer=cleaning).fit(df['Sentence'])
bow_transform=bow_fit.transform(df['Sentence'])

In [64]:
print(bow_transform.toarray())

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [67]:
from sklearn.feature_extraction.text import TfidfTransformer
transformed=TfidfTransformer().fit(bow_transform)
tfid4=transformed.transform(bow_transform)
print(tfid4)

  (0, 13572)	0.15215533441759058
  (0, 13163)	0.15215533441759058
  (0, 12533)	0.22823304857446028
  (0, 12279)	0.19336716738204976
  (0, 12094)	0.2483059166321302
  (0, 11559)	0.12251353725517967
  (0, 11498)	0.2483059166321302
  (0, 11431)	0.20369612678627808
  (0, 11127)	0.2134400354397197
  (0, 11051)	0.23997492367012072
  (0, 8843)	0.19195425169061767
  (0, 8684)	0.18576935351018317
  (0, 7043)	0.07236980564886508
  (0, 6983)	0.19982918755478082
  (0, 6500)	0.2483059166321302
  (0, 5780)	0.2483059166321302
  (0, 4788)	0.26004779172779063
  (0, 3787)	0.2483059166321302
  (0, 3733)	0.22823304857446028
  (0, 2688)	0.26004779172779063
  (0, 2213)	0.21649117347879987
  (0, 2177)	0.2134400354397197
  (1, 12401)	0.31570243953538835
  (1, 12072)	0.41941479302355916
  (1, 11183)	0.35160442494224164
  :	:
  (5840, 10289)	0.36884836617186495
  (5840, 9068)	0.250167993662527
  (5840, 8355)	0.27860280346703287
  (5840, 4170)	0.2193076682223426
  (5840, 3124)	0.1620015038878969
  (5840, 670)	0.

In [71]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(df['Sentence'],df['Sentiment'],train_size=0.20,random_state=234)

In [74]:
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC

In [75]:
pipeline=Pipeline([
    ('vectorizer',CountVectorizer(analyzer=cleaning)),
    ("normalizer",TfidfTransformer()),
    ('Clf',SVC())
])

In [77]:
pipeline

Pipeline(steps=[('vectorizer',
                 CountVectorizer(analyzer=<function cleaning at 0x0000018D86803740>)),
                ('normalizer', TfidfTransformer()), ('Clf', SVC())])

In [78]:
pipeline.fit(x_train,y_train)

Pipeline(steps=[('vectorizer',
                 CountVectorizer(analyzer=<function cleaning at 0x0000018D86803740>)),
                ('normalizer', TfidfTransformer()), ('Clf', SVC())])

In [82]:
predicted=pipeline.predict(x_test)

In [87]:
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report

In [88]:
print(accuracy_score(y_test,predicted))
print(confusion_matrix(y_test,predicted))
print(classification_report(y_test,predicted))

0.5755241762943946
[[  12  608   71]
 [  74 2364   59]
 [   5 1167  314]]
              precision    recall  f1-score   support

    negative       0.13      0.02      0.03       691
     neutral       0.57      0.95      0.71      2497
    positive       0.71      0.21      0.33      1486

    accuracy                           0.58      4674
   macro avg       0.47      0.39      0.36      4674
weighted avg       0.55      0.58      0.49      4674

